In [ ]:
ShmekelPath = r'C:\Users\Danielle\PycharmProjects\Shmekel'
template_path = r'configs/dror/preprocess/e01_dense_model/template.json'

import os
os.chdir(ShmekelPath)

import pandas
import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)
from pandas.api.types import is_string_dtype, is_numeric_dtype
import numpy as np
from api.core.grid_search import GridSearch


from Utils.logger import logger
logger.set_stream_level('warning')


gs = GridSearch(template_path=template_path)


def _generate_plotly_dim_dict(df: pandas.DataFrame, field):
    dim_dict = {}
    dim_dict["label"] = field
    column = df[field]
    if is_numeric_dtype(column):
        dim_dict["values"] = column
    elif is_string_dtype(column):
        texts = column.unique()
        dim_dict["values"] = [
            np.argwhere(texts == x).flatten()[0] for x in column
        ]
        dim_dict["tickvals"] = list(range(len(texts)))
        dim_dict["ticktext"] = texts
    else:
        raise Exception("Unidentifiable Type")

    return dim_dict


def data_frame_to_parallel_plot_data(df: pandas.DataFrame):
    line = dict(color=-df['val_acc'], colorscale='Jet')
    dimensions = [_generate_plotly_dim_dict(df, field) for field in df]

    return [go.Parcoords(line=line, dimensions=dimensions)]


def grid_search_to_data_frame(gs: GridSearch, metrics=('val_acc',)):
    converter = gs.get_exp_name_2_combs_converter(minimal=True)

    columns = None
    for comb in converter.values():
        columns = [field for field in comb.keys()]
        break

    columns = ('__Name__',) +  tuple([c for c in columns if c not in ['__Name__']]) + tuple(metrics)

    df = {field: [] for field in columns}

    for exp in gs:
        line = converter[exp.name]

        for m in metrics:
            line[m] = max(exp.results[m])

        for field, val in line.items():
            if isinstance(val, list):
                val = str(val)
            
            df[field].append(val)
    
#     _ = df.pop('__InputShape__')
    
    return pandas.DataFrame.from_dict(df)

In [ ]:
plotly.offline.iplot(data_frame_to_parallel_plot_data(grid_search_to_data_frame(gs)))